<a href="https://colab.research.google.com/github/PoojaDurai97/num1/blob/master/AdvNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [ ]:
seed_urls = ['https://inshorts.com/en/read/technology',
             'https://inshorts.com/en/read/sports',
             'https://inshorts.com/en/read/world']

def build_dataset(seed_urls):
    news_data = []
    for url in seed_urls:
        news_category = url.split('/')[-1]
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        
        news_articles = [{'news_headline': headline.find('span', 
                                                         attrs={"itemprop": "headline"}).string,
                          'news_article': article.find('div', 
                                                       attrs={"itemprop": "articleBody"}).string,
                          'news_category': news_category}
                         
                            for headline, article in 
                             zip(soup.find_all('div', 
                                               class_=["news-card-title news-right-box"]),
                                 soup.find_all('div', 
                                               class_=["news-card-content news-right-box"]))
                        ]
        news_data.extend(news_articles)
        
    df =  pd.DataFrame(news_data)
    df = df[['news_headline', 'news_article', 'news_category']]
    return df

In [ ]:
news_df = build_dataset(seed_urls)
news_df.head()

,news_headline,news_article,news_category
0,Bose launches QC Earbuds with 'world’s most ef...,Bose has launched the new QuietComfort Earbuds...,technology
1,WhatsApp launches feature to make messages dis...,WhatsApp on Thursday launched 'disappearing me...,technology
2,WhatsApp allowed to offer payments service via...,The National Payments Corporation of India on ...,technology
3,NPCI imposes 30% cap for third-party apps offe...,The NPCI imposed a cap on the share of UPI tra...,technology
4,Twitter labels Trump's tweet accusing Democrat...,US President Donald Trump accused Democrats of...,technology


In [ ]:
page = requests.get('https://www.ndtv.com/india-news/details-of-dissent-letter-to-sonia-gandhi-steady-decline-no-honest-inspection-2286399') 
soup = BeautifulSoup(page.content, 'html.parser')



In [ ]:
for s in soup.select('script'):
    s.extract()
for a in soup.findAll('a', href=True):
    a.extract()

In [ ]:
for tags in soup.findAll("p", class_=["live_tv","ft-social--txt"]):
    tags.extract()

In [ ]:

for tags in soup.findAll("div", class_=["watch-news_inner","ins_adwrap marginb20"]):
    tags.extract()
for div in soup.findAll("div", {'id':'jiosaavn-widget'}): 
    div.decompose()

In [ ]:
soup

<!DOCTYPE doctype html>
<html itemscope="" itemtype="http://schema.org/NewsArticle" xmlns="http://www.w3.org/1999/xhtml"><head><title>Dissent Letter To Sonia Gandhi: No More A Nurturing Leadership</title><meta content="Sonia Gandhi,Rahul Gandhi,Congress" itemprop="keywords" name="news_keywords"/><meta content='The letter by 23 Congress dissidents to party chief Sonia Gandhi says there has been a "steady decline of the party", witnessed in the 2014 and 2019 elections, but there has been no honest introspection to analyse the reasons.' itemprop="description" name="description"/><meta content="india" itemprop="articleSection" name="section"/><meta content="https://www.ndtv.com/india-news/details-of-dissent-letter-to-sonia-gandhi-steady-decline-no-honest-inspection-2286399" itemprop="url" name="url"/><link href="https://www.ndtv.com/india-news/details-of-dissent-letter-to-sonia-gandhi-steady-decline-no-honest-inspection-2286399?amp=1&amp;akamai-rum=off" rel="amphtml"/><link href="https://p

In [ ]:
p_tags = soup.find_all(['p','ul'])
s=[]
for each in p_tags:
    print (each.text)
    s.append(each.text)









The letter said it would "always remain an integral part of the collective leadership (File)
The letter by 23 Congress dissidents to party chief Sonia Gandhi says there has been a "steady decline of the party", witnessed in the 2014 and 2019 elections, but there has been no honest introspection to analyse the reasons. It also says the Congress is "no more a nurturing leadership" and calls for "free, fair and democratic" elections.
"We have witnessed a steady decline of the Congress Party reflected in successive electoral verdicts in States and General Elections in 2014 and 2019. Reasons are manifold and need to be immediately identified. Otherwise, the Congress Party will find itself marginalized, both in the States, which is already apparent, as well as at the national level," says the letter accessed by NDTV.
"The erosion of support base and, in particular, losing the confidence of the youth, is a matter of serious concern," it says.
Veteran leaders Ghulam Nabi Azad, Kapil Si

In [ ]:
soup.title.text

'Dissent Letter To Sonia Gandhi: No More A Nurturing Leadership'

In [ ]:
listToStr = ' '.join([str(elem) for elem in s]) 
listToStr=listToStr.replace("       ", "") 
listToStr

' The letter said it would "always remain an integral part of the collective leadership (File) The letter by 23 Congress dissidents to party chief Sonia Gandhi says there has been a "steady decline of the party", witnessed in the 2014 and 2019 elections, but there has been no honest introspection to analyse the reasons. It also says the Congress is "no more a nurturing leadership" and calls for "free, fair and democratic" elections. "We have witnessed a steady decline of the Congress Party reflected in successive electoral verdicts in States and General Elections in 2014 and 2019. Reasons are manifold and need to be immediately identified. Otherwise, the Congress Party will find itself marginalized, both in the States, which is already apparent, as well as at the national level," says the letter accessed by NDTV. "The erosion of support base and, in particular, losing the confidence of the youth, is a matter of serious concern," it says. Veteran leaders Ghulam Nabi Azad, Kapil Sibal, A

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

In [ ]:

from transformers import T5Tokenizer, T5ForConditionalGeneration

#tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = T5ForConditionalGeneration.from_pretrained("/content/gdrive/My Drive/my_t5_model/")
tokenizer = T5Tokenizer.from_pretrained("/content/gdrive/My Drive/my_t5_tokenizer/")

def summarize(text, max_length=150):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]

In [ ]:
l1=summarize(listToStr,150)
len(l1.split())


Token indices sequence length is longer than the specified maximum sequence length for this model (852 > 512). Running this sequence through the model will result in indexing errors


52

In [ ]:
l1

'"the uncertainty over the leadership and the drift, has demoralised the Congress workers and further weakened the party," it added. The letter also called for an "institutional leadership mechanism" to collectively guide the party\'s revival. It further said there has been no honest introspection to analyse the reasons for its continued decline.'

In [ ]:

page = requests.get('https://scroll.in/latest/971665/letter-to-sonia-gandhi-kerala-congress-chief-steps-in-after-state-legislator-attacks-shashi-tharoor') 
soup = BeautifulSoup(page.content, 'html.parser')



In [ ]:
for s in soup.select('script'):
    s.extract()
for a in soup.findAll('a', href=True):
    a.extract()

In [ ]:
p_tags = soup.find_all(['p','ul'])
s=[]
for each in p_tags:
    print (each.text)
    s.append(each.text)

































































































































































































































































Kerala Congress chief  on Friday stepped in to stop an attack on party colleague Shashi Tharoor over the “dissent letter” written to Congress President Sonia Gandhi, the Hindustan Times reported. Tharoor was one of the 23 signatories to the letter. The letter had called for radical changes within the party, pointing to its dismal electoral performances and shrinking voter base.
The controversy began when Kerala Congress leader Kodikunnil Suresh called Tharoor a “guest artist and immature politician”. Radhakrishnan, who had himself criticised Tharoor over the letter, then stepped in to stop the infighting. He asked all leaders to treat the matter as closed and not air opinions freely.
“Tharoor is not a politician,” Suresh said. “For th

In [ ]:
listToStr2= ' '.join([str(elem) for elem in s]) 
#listToStr=listToStr.replace("       ", "") 


In [ ]:
listToStr2.strip()

'Kerala Congress chief  on Friday stepped in to stop an attack on party colleague Shashi Tharoor over the “dissent letter” written to Congress President Sonia Gandhi, the Hindustan Times reported. Tharoor was one of the 23 signatories to the letter. The letter had called for radical changes within the party, pointing to its dismal electoral performances and shrinking voter base. The controversy began when Kerala Congress leader Kodikunnil Suresh called Tharoor a “guest artist and immature politician”. Radhakrishnan, who had himself criticised Tharoor over the letter, then stepped in to stop the infighting. He asked all leaders to treat the matter as closed and not air opinions freely. “Tharoor is not a politician,” Suresh said. “For the party, he’s only a guest artist. He is immature and yet to learn to work well within the party framework. It is not right for him to take different positions thinking that he is a global citizen.” Senior leader K Muralidharan also lashed out at Tharoor,

In [ ]:
l2=summarize(listToStr2,150)
l2

Token indices sequence length is longer than the specified maximum sequence length for this model (696 > 512). Running this sequence through the model will result in indexing errors


'chief Kodikunnil Suresh on Friday stepped in to stop an attack on party colleague Shashi Tharoor over the “dissent letter” written to Congress President Sonia Gandhi. Tharoor was one of the 23 signatories to the letter, which called for radical changes within the party. Notably, Tharoor is not a politician, but a guest artist and yet to learn to work well within the party framework.'

In [ ]:
pip install -U sentence-transformers


     |████████████████████████████████| 71kB 5.9MB/s 
     |████████████████████████████████| 1.1MB 8.9MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
     |████████████████████████████████| 1.1MB 46.7MB/s 
     |████████████████████████████████| 3.0MB 55.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=26039fcc151eb551813fdee12440650547a61a7150ebaa0d4b31ce7ae146031e
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c09bbb6c10e3cccac900b260a0b85d86b76b3b0cef25a2f44e9026995ee953d9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/content/gdrive/My Drive/output_albert/training_nli_albert-base-v2-2020-11-05_02-56-09/')

In [ ]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')
import gensim
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True) 

[=-------------------------------------------------] 2.3% 38.8/1662.8MB downloaded

In [ ]:
sentence_obama = 'Obama speaks to the media in Illinois'
sentence_president = 'The president greets the press in Chicago'
model.wmdistance(sentence_obama, sentence_president)

ModuleAttributeError: ignored

In [ ]:
e1=model.encode([l1])
e2=model.encode([l3])

In [ ]:
e2.shape

(1, 768)

In [ ]:

def run_sts_benchmark(session):
  """Returns the similarity scores"""
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={})
  return scores

def pearson_and_spearman(preds, labels):
        pearson_corr = pearsonr(preds, labels)[0]
        spearman_corr = spearmanr(preds, labels)[0]
        return {
            "pearson": pearson_corr,
            "spearmanr": spearman_corr,
            "corr": (pearson_corr + spearman_corr) / 2,
        }


In [ ]:

# For evaluation we use exactly normalized rather than
# approximately normalized.
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.stats
import pandas as pd
from scipy.stats import pearsonr, spearmanr
sts_encode1 = tf.nn.l2_normalize(e1, axis=1)
sts_encode2 = tf.nn.l2_normalize(e2, axis=1)
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)




with tf.Session() as session:
 session.run(tf.global_variables_initializer())
 session.run(tf.tables_initializer())
 scores = run_sts_benchmark(session)


In [ ]:
import math
def angular_distance(src, tgt):
    cos_sim = cosine_similarity(src, tgt)
    np.fill_diagonal(cos_sim, 1)
    distance_ = 1 - (np.arccos(cos_sim) / math.pi)
    return distance_
angular_distance(e1, e2)  

array([[1.]], dtype=float32)

In [ ]:
def manhattan_dist(src, tgt):
    distance_ = manhattan_distances(src, tgt)
    return distance_
from sklearn.metrics.pairwise import (
    cosine_similarity,
    euclidean_distances,
    manhattan_distances,
)

manhattan_dist(e1, e2)

array([[692.03031792]])

In [ ]:
def euclidean_dist(src, tgt):
    distance_ = euclidean_distances(src, tgt)
    return distance_
euclidean_dist(e1, e2)

array([[32.878075]], dtype=float32)

In [ ]:
def inner_product(src, tgt):
    similarity_ = np.inner(src, tgt)
    return similarity_
inner_product(e1, e2)

array([[796.4137]], dtype=float32)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
get_cosine_similarity(e1, e2)

0.84524596

In [ ]:
from numpy.linalg import norm




def theta(src, tgt):
    similarity_ = cosine_sim(src, tgt)
    np.fill_diagonal(similarity_, 1)
    theta_ = np.arccos(similarity_) + math.radians(10)
    return theta_


def magnitude_difference(src, tgt):
    src_len = len(src)
    src_norm = norm(src, axis=1).repeat(src_len).reshape(src_len, -1)
    tgt_norm = norm(tgt, axis=1)
    difference_ = np.abs(src_norm - tgt_norm)
    return difference_


def triangle_area_similarity(src, tgt, theta_):
    src_len = len(src)
    src_norm = norm(src, axis=1).repeat(src_len).reshape(src_len, -1)
    tgt_norm = norm(tgt, axis=1)
    triangle_similarity_ = (src_norm * tgt_norm * np.sin(theta_)) / 2
    return triangle_similarity_


def sector_area_similarity(src, tgt, theta_):
    distance_ = euclidean_dist(src, tgt)
    difference_ = magnitude_difference(src, tgt)
    sector_similarity_ = math.pi * ((distance_ + difference_) ** 2)
    sector_similarity_ *= theta_ / 360
    return sector_similarity_


def triangle_sector_similarity(src, tgt):
    theta_ = theta(src, tgt)
    triangle_similarity_ = triangle_area_similarity(src, tgt, theta_)
    sector_similarity_ = sector_area_similarity(src, tgt, theta_)
    ts_ss_ = triangle_similarity_ * sector_similarity_
    return ts_ss_

In [ ]:

page = requests.get('https://www.republicworld.com/india-news/politics/dissenting-letter-brainchild-of-2-congress-functionaries-who-are-not-s.html') 
soup = BeautifulSoup(page.content, 'html.parser')



In [ ]:
for s in soup.select('script'):
    s.extract()
for a in soup.findAll('a', href=True):
    a.extract()

for tags in soup.findAll("p", class_=["ham-title","mrgn0 color-white","mrgn0 font12"]):
    tags.extract()

In [ ]:
p_tags = soup.find_all(['p','ul'])
s=[]
for each in p_tags:
    print (each.text)
    s.append(each.text)


On Monday, top sources told Republic TV that the letter of dissent by 23 senior Congress leaders is the brainchild of two party functionaries who are not signatories to the letter. The two Congress leaders who have facilitated the dissenting letter have distanced themselves from either camp for now. Sources added that one of the aforesaid leaders is not even present in Delhi. Reportedly, the leaders behind the letter are pretending to be neutral. It is perceived that the fight between the two camps is expected to escalate.

Reportedly, several Congress leaders wrote to the Congress president on August 12 calling for sweeping changes in the party's functioning. The 23 leaders who have signed the letter are Leader of Opposition in Rajya Sabha Ghulam Nabi Azad, Anand Sharma, Kapil Sibal, Manish Tewari, Shashi Tharoor, Vivek Tankha, M Veerappa Moily, Mukul Wasnik, Jitin Prasada, Bhupinder Singh Hooda, Rajinder Kaur Bhattal, Prithviraj Chavan,  PJ Kurian, Ajay Singh, Renuka Chaudhary, Milin

In [ ]:
listToStr3= ' '.join([str(elem) for elem in s]) 
listToStr3 = listToStr3.replace(u'\xa0', u' ')

In [ ]:
listToStr3

"On Monday, top sources told Republic TV that the letter of dissent by 23 senior Congress leaders is the brainchild of two party functionaries who are not signatories to the letter. The two Congress leaders who have facilitated the dissenting letter have distanced themselves from either camp for now. Sources added that one of the aforesaid leaders is not even present in Delhi. Reportedly, the leaders behind the letter are pretending to be neutral. It is perceived that the fight between the two camps is expected to escalate.  Reportedly, several Congress leaders wrote to the Congress president on August 12 calling for sweeping changes in the party's functioning. The 23 leaders who have signed the letter are Leader of Opposition in Rajya Sabha Ghulam Nabi Azad, Anand Sharma, Kapil Sibal, Manish Tewari, Shashi Tharoor, Vivek Tankha, M Veerappa Moily, Mukul Wasnik, Jitin Prasada, Bhupinder Singh Hooda, Rajinder Kaur Bhattal, Prithviraj Chavan,  PJ Kurian, Ajay Singh, Renuka Chaudhary, Mili

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("/content/gdrive/My Drive/my_t5_model/")
tokenizer = T5Tokenizer.from_pretrained("/content/gdrive/My Drive/my_t5_tokenizer/")
l3=summarize(listToStr3,150)
l3

Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors


'a letter of dissent by 23 senior Congress leaders is the brainchild of two party functionaries who are not signatories to the letter. The two party functionaries behind the letter have distanced themselves from either camp for now, according to reports.'

In [ ]:

page = requests.get('https://www.timesnownews.com/sports/cricket/article/chennai-super-kings-vs-mumbai-indians-head-to-head-record-important-stats-ahead-of-csks-must-win-tie/671418') 
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
for s in soup.select('script'):
    s.extract()
for a in soup.findAll('a', href=True):
    a.extract()


In [ ]:
p_tags = soup.find_all(['p'])
s=[]
for each in p_tags:
    print (each.text)
    s.append(each.text)


With an aim to keep for their playoffs hopes alive, Chennai Super Kings (CSK) will face defending champions, Mumbai Indians (MI), in the 41st match of the Indian Premier League (IPL) 2020. Both the teams have had contrasting fortunes so far. With just three wins from 10 matches, CSK are reeling at the last spot in the points table. On the other hand, MI, who have also achieved the top spot on many occasions, currently sit at the third spot with six wins from nine matches. A win here will see them regain the numero uno position.
For the first time in the history of IPL, CSK are staring at an early exit. The only team to qualify for playoffs in every season they have been part of, CSK had kicked off their season with a win. After defeating Mumbai Indians in the season-opener, things haven't gone their way and the Yellow Army has only managed two more wins.
A win is the need of the hour for CSK and a defeat here will end their campaign. In past, teams with six wins have made it to the pla

In [ ]:
listToStr4= ' '.join([str(elem) for elem in s]) 

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("/content/gdrive/My Drive/my_t5_model/")
tokenizer = T5Tokenizer.from_pretrained("/content/gdrive/My Drive/my_t5_tokenizer/")
l4=summarize(listToStr4,150)
l4

Token indices sequence length is longer than the specified maximum sequence length for this model (673 > 512). Running this sequence through the model will result in indexing errors


"defending champions, Mumbai Indians (MI), in the 41st match of the Indian Premier League (IPL) 2020. Chennai Super Kings have won just three out of 10 matches so far this season and are currently sitting at the third spot with 627 runs against CSK. However, MI's skipper MS Dhoni sits at the top with 627 runs while Lasith Malinga comes at the top with 13 scalps."

In [ ]:

page = requests.get('https://www.india.com/news/india/air-pollution-ahead-of-diwali-these-states-ban-use-and-sale-of-fireworks-complete-list-here-4196431/') 
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
for s in soup.select('script'):
    s.extract()
for a in soup.findAll('a', href=True):
    a.extract()


In [ ]:
p_tags = soup.find_all(['p'])
s=[]
for each in p_tags:
    print (each.text)
    s.append(each.text)


Air Pollution Latest News: A day after the National Green Tribunal issued notice to Ministry of Environment and Forests (MoEF) to ban the use of fire crackers from November 7-30 in the interest of public health and environment, a number of states have issued order in this regard. Starting from Rajasthan and Odisha to Bengal and Haryana — these states have made it mandatory to ban the use of fire crackers, keeping in mind the pollution and coronavirus pandemic in mind. Also Read - 
Rajasthan Also Read - 
The Rajasthan government has decided to ban the sale of firecrackers during the festive season in the state in the wake of the crisis created by the coronavirus pandemic. “Protecting the lives of people is paramount for the government in this challenging time,” Chief Minister Ashok Gehlot said. Also Read - 
He further added that in a situation like this, people should avoid using fireworks on Diwali.
Delhi
Delhi Environment Minister Gopal Rai on Tuesday urged the people of Delhi to avoi

In [ ]:
listToStr5= ' '.join([str(elem) for elem in s]) 

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("/content/gdrive/My Drive/my_t5_model/")
tokenizer = T5Tokenizer.from_pretrained("/content/gdrive/My Drive/my_t5_tokenizer/")
l5=summarize(listToStr5,150)
l5

Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors


'from Rajasthan and Odisha to Bengal and Haryana have made it mandatory to ban the use of firecrackers during Diwali, keeping in mind the pollution and coronavirus pandemic in mind. Notably, the National Green Tribunal issued notice to Ministry of Environment and Forests (MoEF) to ban the use of firecrackers from November 7-30 in the interest of public health and environment. The NGT also appointed senior advocate Raj Panjwani and advocate Shibhani Ghosh to assist it.'